In [7]:
import pandas as pd
import os


DATA_DIR = '../data/seattle'

df_calendar = pd.read_parquet(os.path.join(DATA_DIR, 'calendar.parquet'))
df_listings = pd.read_parquet(os.path.join(DATA_DIR, 'listings.parquet'))
df_reviews = pd.read_parquet(os.path.join(DATA_DIR, 'reviews.parquet'))

display('df_calendar',df_calendar.head(2))
display('df_listings',df_listings.head(2)) 
display('df_reviews',df_reviews.head(2))

'df_calendar'

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,9419,2024-12-23,f,$62.00,NaN,2,90
1,9419,2024-12-24,f,$62.00,NaN,2,90


'df_listings'

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6606,https://www.airbnb.com/rooms/6606,20241223022600,2024-12-23,city scrape,"Fab, private seattle urban cottage!","This tiny cottage is only 15x10, but it has ev...","A peaceful yet highly accessible neighborhood,...",https://a0.muscache.com/pictures/45742/21116d7...,14942,...,4.77,4.88,4.57,str-opli-19-002622,f,2,2,0,0,0.86
1,9419,https://www.airbnb.com/rooms/9419,20241223022600,2024-12-23,city scrape,Glorious sun room w/ memory foambed,This beautiful double room features sun filled...,"Lots of restaurants (see our guide book) bars,...",https://a0.muscache.com/pictures/56645186/e5fb...,30559,...,4.89,4.68,4.67,Exempt,f,11,0,11,0,1.19


'df_reviews'

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,9419,71022,2010-07-30,166478,Beth,"If you love art, animals, and yoga, this is th..."
1,9419,83099,2010-08-23,165949,Erica,I only overnighted here but I loved the offbea...


In [ ]:
!which python # Should be /Users/aus10powell/opt/anaconda3/bin/python

## Initial Ranker Example


In [2]:
# import importlib
# importlib.reload(ListingRanker)

In [8]:
import sys
sys.path.append('../src')
import listing_ranker
import importlib
importlib.reload(listing_ranker)
from listing_ranker import ListingRanker

# Initialize ranker with data
ranker = ListingRanker(listings_df=listings_df, reviews_df=reviews_df)

# Example: Get recommendations for a user
user_id = reviews_df['reviewer_id'].iloc[0]  # Get first user as example
print(f"User ID: {user_id}")
print(f"User history: {reviews_df[reviews_df['reviewer_id'] == user_id]['listing_id'].tolist()}")
user_history = listings_df[listings_df['listing_id'].isin(
    reviews_df[reviews_df['reviewer_id'] == user_id]['listing_id']
)]

# Get candidate items (excluding user history)
candidates = listings_df[~listings_df['listing_id'].isin(user_history['listing_id'])]

# Get recommendations
recommendations = ranker.retrieve_candidates(
    user_history=user_history,
    candidates=candidates,
    interaction_data=reviews_df,
    top_k=5
)

print("\nTop 5 recommendations:")
print(recommendations)


Found 2 reviews for user 166478


TypeError: ListingRanker.__init__() got an unexpected keyword argument 'model'

### Evaluator 

In [1]:
import os
import sys
notebook_dir = os.path.abspath(os.path.dirname('__file__'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

from src.listing_ranker import ListingRanker
from src.recommender_evaluator import RecommenderEvaluator
import pandas as pd
# Set up data directory
DATA_DIR = '../data/seattle'

# Load the data
df_calendar = pd.read_parquet(os.path.join(DATA_DIR, 'calendar.parquet'))
df_listings = pd.read_parquet(os.path.join(DATA_DIR, 'listings.parquet'))
df_reviews = pd.read_parquet(os.path.join(DATA_DIR, 'reviews.parquet'))

# Initialize the ListingRanker
# Initialize the ranker and evaluator
ranker = ListingRanker(model='phi3')
evaluator = RecommenderEvaluator(
    ranker=ranker,
    df_listings=df_listings,
    df_reviews=df_reviews
)

# Prepare holdout with smaller test size
holdout_data = evaluator.prepare_holdout_set(
    test_size=0.1,  # 10% of users
    min_reviews=3
)

# Run evaluation with small sample
results = evaluator.evaluate(
    holdout_data=holdout_data,
    k=5,  # Start with top-5 recommendations
    sample_size=5  # Test with just 5 users first
)

# Print results
print("\nEvaluation Results:")
for metric, value in results['mean_metrics'].items():
    print(f"{metric}: {value:.4f}")

# Optionally plot results
evaluator.plot_results(pd.DataFrame([results['mean_metrics']]))

/Users/aus10powell/Documents/Projects/AirRanker/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'matplotlib'

In [21]:

# Process results - ensure we're only averaging numeric values
processed_results = {}
for metric, values in results.items():
    if isinstance(values, (list, np.ndarray)):  # If it's a list of values
        try:
            # Convert to float if possible and calculate mean
            numeric_values = [float(v) for v in values if str(v).replace('.','',1).isdigit()]
            if numeric_values:
                processed_results[metric] = np.mean(numeric_values)
            else:
                processed_results[metric] = 0.0
        except (ValueError, TypeError):
            # If conversion fails, skip this metric
            continue
    elif isinstance(values, (int, float)):  # If it's already a single numeric value
        processed_results[metric] = values
    else:
        # Store non-numeric values as is (like timestamp)
        processed_results[metric] = values

# Add metadata
processed_results['timestamp'] = "now" #datetime.now().strftime("%Y-%m-%d %H:%M:%S")
processed_results['k'] = 10
processed_results['sample_size'] = 20

# Print results
print("\nEvaluation Results:")
for metric, value in processed_results.items():
    if metric not in ['timestamp', 'k', 'sample_size']:
        if isinstance(value, (int, float)):
            print(f"{metric}: {value:.4f}")
        else:
            print(f"{metric}: {value}")

# Plot results - only include numeric metrics
numeric_results = {k: v for k, v in processed_results.items() 
                  if isinstance(v, (int, float)) and k not in ['k', 'sample_size']}
#evaluator.plot_results(pd.DataFrame([numeric_results]))


Evaluation Results:
ndcg: 0.0000
precision: 0.0000
recall: 0.0000
diversity: 0.7366
coverage: 0.0449
latency: 9.1399


In [15]:
# Create multiple rankers to compare
models_dict = {
    'phi3': ListingRanker(model='phi4'),
    'random': RandomRanker(),
    'popularity': PopularityRanker(df_reviews)
}

# Compare the models
comparison_results = evaluator.compare_models(
    models_dict=models_dict,
    holdout_data=holdout_data,
    k=10,
    sample_size=20
)

# Plot the comparison
evaluator.plot_results(comparison_results)

NameError: name 'RandomRanker' is not defined